# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [64]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [65]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port lincoln,-34.7333,135.8667,12.75,62,2,9.61,AU,1724792613
1,1,luderitz,-26.6481,15.1594,13.40,80,0,7.24,NaN,1724792615
2,2,robertson,36.5167,-86.8667,37.51,31,0,2.57,US,1724792616
3,3,capinota,-17.7167,-66.2333,25.06,21,1,6.52,BO,1724792617
4,4,vilyuchinsk,52.9306,158.4028,12.15,83,100,1.36,RU,1724792618


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [66]:
#%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humdidity",
    color = "Red"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [67]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal_cities = city_data_df.loc[(city_data_df['Max Temp']<20) & (city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness']>10), :]

# Drop any rows with null values
#my_ideal_cities = my_ideal_cities.dropna()
# Display sample data
my_ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,vilyuchinsk,52.9306,158.4028,12.15,83,100,1.36,RU,1724792618
7,7,abbeville,50.1000,1.8333,16.38,69,30,3.01,FR,1724792622
10,10,taro,-8.3862,115.2800,17.57,95,100,0.52,ID,1724792625
12,12,bilibino,68.0546,166.4372,3.46,85,100,2.45,RU,1724792628
13,13,waitangi,-43.9535,-176.5597,12.12,98,100,4.47,NZ,1724792629
16,16,blackmans bay,-43.0167,147.3167,7.07,91,73,0.89,AU,1724792633
17,17,iqaluit,63.7506,-68.5145,6.85,100,100,2.06,CA,1724792634
18,18,margaret river,-33.9500,115.0667,13.47,67,100,4.33,AU,1724792636
24,24,talnakh,69.4865,88.3972,12.39,97,99,1.25,RU,1724792645
30,30,grytviken,-54.2811,-36.5092,3.20,83,91,4.27,GS,1724792653


### Step 3: Create a new DataFrame called `hotel_df`.

In [68]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,vilyuchinsk,RU,52.9306,158.4028,83,
7,abbeville,FR,50.1000,1.8333,69,
10,taro,ID,-8.3862,115.2800,95,
12,bilibino,RU,68.0546,166.4372,85,
13,waitangi,NZ,-43.9535,-176.5597,98,
16,blackmans bay,AU,-43.0167,147.3167,91,
17,iqaluit,CA,63.7506,-68.5145,100,
18,margaret river,AU,-33.9500,115.0667,67,
24,talnakh,RU,69.4865,88.3972,97,
30,grytviken,GS,-54.2811,-36.5092,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
"categories":categories,
"limit":1,
"apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    my_row_lat = row["Lat"]
    my_row_lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    filters = f"circle:{my_row_lon},{my_row_lat},{radius}"
    bias = f"proximity:{my_row_lon},{my_row_lat}"
    params["filter"] = filters
    params["bias"] = bias


    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vilyuchinsk - nearest hotel: Победа
abbeville - nearest hotel: Hôtel Mercure
taro - nearest hotel: Padma Resort Ubud
bilibino - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
blackmans bay - nearest hotel: Villa Howden
iqaluit - nearest hotel: Frobisher Inn
margaret river - nearest hotel: Margaret River Hotel
talnakh - nearest hotel: Талнах
grytviken - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,vilyuchinsk,RU,52.9306,158.4028,83,Победа
7,abbeville,FR,50.1000,1.8333,69,Hôtel Mercure
10,taro,ID,-8.3862,115.2800,95,Padma Resort Ubud
12,bilibino,RU,68.0546,166.4372,85,No hotel found
13,waitangi,NZ,-43.9535,-176.5597,98,Hotel Chathams
16,blackmans bay,AU,-43.0167,147.3167,91,Villa Howden
17,iqaluit,CA,63.7506,-68.5145,100,Frobisher Inn
18,margaret river,AU,-33.9500,115.0667,67,Margaret River Hotel
24,talnakh,RU,69.4865,88.3972,97,Талнах
30,grytviken,GS,-54.2811,-36.5092,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
#%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
"Lng",
"Lat",
geo = True,
tiles = "EsriNatGeo",
frame_width = 700,
frame_height = 500,
size = "Humdidity",
color = "red"
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]